In [1]:
import pandas as pd

In [2]:
#import datasets
text = pd.read_csv("../Data Preprocessing/fta_clean.csv")
data = pd.read_csv('../Data Preprocessing/trade_clean.csv')
text = text.drop(138)
text = text.set_index('Identifier')

In [3]:
supervised = pd.read_csv('../Data Labelling/machine_labelled.csv', index_col='Identifier')
supervised = supervised.drop(171)

In [4]:
#convert column to datetime
text['Date signed'] = pd.to_datetime(text['Date signed'])

In [5]:
text['Date signed'] = text['Date signed'].dt.year

In [6]:
columns = ['iso3num_o', 'iso3num_d', 'contig', 'dist', 'comlang_off',
       'comlang_ethno', 'comcol', 'comrelig', 'comleg_pretrans',
       'comleg_posttrans', 'col_dep_ever', 'sibling_ever', 'gdp_o', 'gdp_d',
        'gdpcap_o','gdp_ppp_d', 'gdpcap_ppp_o', 'entry_cost_o', 'entry_cost_d',
       'entry_tp_o', 'entry_tp_d', 'tradeflow_imf_o', 'tradeflow_imf_d']

#merge the gravity dataset with the texts dataset on the rows of the texts dataset
def get_dat(row):
    return data[(data['year'] == row['Date signed']) & (data['iso3_o'] == row['Party 1']) & (data['iso3_d'] == row['Party 2'])].iloc[0][columns]

merged = pd.concat([text,supervised,text.apply(get_dat, axis=1)], axis=1)


In [7]:
merged = merged.drop(['Text', 'Articles'], axis=1)

In [8]:
#set columns to the data of the following year
cols = ['tradeflow_imf_o', 'tradeflow_imf_d', 'gdp_o', 'gdp_d']
data = data.set_index("year")

In [9]:
def get_dat_next(row):
    return data.shift(periods = -1)[(data.index == row['Date signed']) & (data['iso3_o'] == row['Party 1']) & (data['iso3_d'] == row['Party 2'])].iloc[0][cols]

merged[["tradeflow_imf_o_next", "tradeflow_imf_d_next", "gdp_o_next", "gdp_d_next"]] = text.apply(get_dat_next, axis = 1)

In [10]:
#get identifiers from another dataset
df = pd.read_csv("../Data Preprocessing/merged.csv")
df = df.set_index("Identifier")

In [11]:
cols = ["Class 0", "Class 1", "Class 2"]
merged = pd.concat([merged, df[cols]], axis = 1)

In [12]:
#remove rows without tradeflow data from either country
merged = merged[~(merged["tradeflow_imf_o"].isna() & merged["tradeflow_imf_d"].isna())]

In [13]:
import numpy as np

In [18]:
#calculate change variables
def change(row):
    c = np.where(np.isnan(row["tradeflow_imf_o"]), row["tradeflow_imf_d"] , row["tradeflow_imf_o"])
    n = np.where(np.isnan(row["tradeflow_imf_o_next"]), row["tradeflow_imf_d_next"] , row["tradeflow_imf_o_next"])
    return 100 * (n - c)/c
    
merged["tradeflow_change"] = merged.apply(change, axis = 1)

In [19]:
merged["gdp_change_o"] = 100 * (merged["gdp_o_next"] - merged["gdp_o"])/merged["gdp_o"]
merged["gdp_change_d"] = 100 * (merged["gdp_d_next"] - merged["gdp_d"])/merged["gdp_d"]

In [20]:
merged["tradeflow_change"]

Identifier
1      15.169746
2       7.887407
3     -15.646390
7       7.343778
9      16.549247
         ...    
435     6.039525
436    16.600019
439     5.269667
442    -4.683943
445   -38.452158
Name: tradeflow_change, Length: 220, dtype: float64

In [21]:
merged.to_csv("predict_tradeflow.csv")